# QRT Portfolio Optimisation

In [126]:
import pandas as pd
import numpy as np
import lseg.data as ld
from pprint import pprint

ld.open_session()

print("Connected to Workspace")

Connected to Workspace


## Fetch the Data
We import the universe that we wish to operate in as well as the various factors that we would like to use

In [127]:
# The universe of equities we will be dealing with
with open('universe.txt') as f:
    universe = f.read().splitlines()

print("Universe of size", len(universe))
print(universe)

field_names = []
fields = {} # fields[name] = {}

class field:
    name = ""
    lag = 0
    string_rep = ""
    
    def __repr__(self):
        return f"ric = {self.name}, lag = {self.lag} ({self.string_rep})"
    
# The various fields we will be using
with open('technical_fields.txt', 'r') as f:
    for line in f:
        field_name, lag = line.split()
        field_names.append(field_name)
        fields[field_name] = field()
        fields[field_name].name = field_name
        fields[field_name].lag = int(lag)

# with open('fundamental_fields.txt', 'r') as f:
#     for line in f:
#         name = line.strip()
#         fields[name] = 0

print("Fields: ", fields.keys())

Universe of size 25
['AAPL.O', 'AMZN.O', 'AVGO.O', 'BAC', 'BRKb', 'COST.O', 'CRM', 'GOOG.O', 'GOOGL.O', 'HD', 'JNJ', 'JPM', 'LLY', 'MA', 'META.O', 'MSFT.O', 'NFLX.O', 'NVDA.O', 'ORCL.K', 'PG', 'TSLA.O', 'UNH', 'V', 'WMT', 'XOM']
Fields:  dict_keys(['TR.TotalReturn1Wk', 'TR.Volume', 'TR.CompanyMarketCapitalization', 'TR.PriceClose', 'TR.PriceOpen'])


In [128]:
start_date = "2024-01-01"
end_date = "2024-03-01"

start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

print("Start Date: ", start_date)
print("End Date: ", end_date)

start_date = start_date.isoformat() + 'Z'
end_date = end_date.isoformat() + 'Z'

Start Date:  2024-01-01 00:00:00
End Date:  2024-03-01 00:00:00


In [129]:
# Get a sample of the data for the universe of equities
data = ld.get_history(universe, list(fields.keys()), start=start_date, end=end_date, interval='1d')
print("Data shape: ", data.shape)
data.head()

Data shape:  (42, 125)


c:\Users\tangy\Desktop\PortfolioManagement\qrtVenv\lib\site-packages\pandas\core\dtypes\cast.py:1056:RuntimeWarning: invalid value encountered in cast
c:\Users\tangy\Desktop\PortfolioManagement\qrtVenv\lib\site-packages\pandas\core\dtypes\cast.py:1080:RuntimeWarning: invalid value encountered in cast


AAPL.O                                          \
           1 Week Total Return    Volume Company Market Capitalization   
Date                                                                     
2024-01-02           -3.838384  82488674                 2870346373440   
2024-01-03           -4.607818  58414460                 2848854338000   
2024-01-04           -6.028515  71983570                 2812673501360   
2024-01-05           -5.895185  62379661                 2801386317280   
2024-01-08           -3.620215  59144470                 2869109421760   

                                               AMZN.O            \
           Price Close Price Open 1 Week Total Return    Volume   
Date                                                              
2024-01-02      185.64     187.15           -2.268431  47339424   
2024-01-03      184.25     184.22           -3.175949  49425495   
2024-01-04      181.91     182.15           -5.743904  56039807   
2024-01-05      181.18     181.99           -4.409635  45153147   
2024-01-08      185.56    182.085           -1.869159  46757053   

                                                                 ...  \
           Company Market Capitalization Price Close Price Open  ...   
Date                                                             ...   
2024-01-02               1556723190000.0      149.93     151.54  ...   
2024-01-03               1541564010000.0      148.47      149.2  ...   
2024-01-04               1501070310000.0      144.57     145.59  ...   
2024-01-05               1508026920000.0      145.24     144.69  ...   
2024-01-08               1548105300000.0       149.1     146.74  ...   

                           WMT                                          \
           1 Week Total Return    Volume Company Market Capitalization   
Date                                                                     
2024-01-02            1.841314  23539700           428845481177.945007   
2024-01-03            0.899417  18756078           428872400822.741028   
2024-01-04            0.120581  19282489            424726363612.35498   
2024-01-05           -0.596258  21709524           421899518224.205017   
2024-01-08             0.38059  20680898           426045555434.591003   

                                                  XOM            \
           Price Close Price Open 1 Week Total Return    Volume   
Date                                                              
2024-01-02   53.096614  52.416614            0.215391  23483049   
2024-01-03   53.099947  53.256613            1.534527  23490779   
2024-01-04   52.586614  53.069947            2.125961  19395154   
2024-01-05   52.236614  52.639947             2.65053  15827392   
2024-01-08   52.749947  52.233281            0.940188  23370127   

                                                                 
           Company Market Capitalization Price Close Price Open  
Date                                                             
2024-01-02                406471560000.0      102.36     100.92  
2024-01-03                409886620000.0      103.22     102.27  
2024-01-04                406312720000.0      102.32     104.08  
2024-01-05                407543730000.0      102.63     103.17  
2024-01-08                400753320000.0      100.92     100.73  

[5 rows x 125 columns]

In [130]:
# Export the data to a CSV file
data.to_csv('data.csv')

In [131]:
for i in range(len(field_names)):
    fields[field_names[i]].string_rep = data.columns[i][1]

pprint(fields)

{'TR.CompanyMarketCapitalization': ric = TR.CompanyMarketCapitalization, lag = 1 (Company Market Capitalization),
 'TR.PriceClose': ric = TR.PriceClose, lag = 2 (Price Close),
 'TR.PriceOpen': ric = TR.PriceOpen, lag = 2 (Price Open),
 'TR.TotalReturn1Wk': ric = TR.TotalReturn1Wk, lag = 0 (1 Week Total Return),
 'TR.Volume': ric = TR.Volume, lag = 1 (Volume)}


## Construct the Dataset
Our data is based on time series. We would do well to append columns of lagged data to the data set to allow the model to refer to past data.

In [132]:
# First we normalize the data by columns
class expanded_feature:
    def __init__(self, name, transform_mean, transform_std):
        self.name = name
        self.transform_mean = transform_mean
        self.transform_std = transform_std
        
    def __repr__(self):
        return f"{self.name} ~ N({self.transform_mean}, {self.transform_std})"
        
expanded_features = {}
for col in data.columns:
    name = col
    mean = data[col].mean()
    std = data[col].std()
    expanded_features[name] = expanded_feature(name, mean, std)

expanded_features

{('AAPL.O',
  '1 Week Total Return'): ('AAPL.O', '1 Week Total Return') ~ N(-0.6356788396880704, 2.9793632889239716),
 ('AAPL.O',
  'Volume'): ('AAPL.O', 'Volume') ~ N(57684892.190476194, 18075954.13049058),
 ('AAPL.O',
  'Company Market Capitalization'): ('AAPL.O', 'Company Market Capitalization') ~ N(2875224079442.381, 63428373302.974434),
 ('AAPL.O',
  'Price Close'): ('AAPL.O', 'Price Close') ~ N(186.12809523809523, 4.130143349602945),
 ('AAPL.O',
  'Price Open'): ('AAPL.O', 'Price Open') ~ N(186.01023809523812, 4.398029762926408),
 ('AMZN.O',
  '1 Week Total Return'): ('AMZN.O', '1 Week Total Return') ~ N(1.6535972445492828, 3.3823211495443477),
 ('AMZN.O',
  'Volume'): ('AMZN.O', 'Volume') ~ N(48360808.85714286, 13781483.730391081),
 ('AMZN.O',
  'Company Market Capitalization'): ('AMZN.O', 'Company Market Capitalization') ~ N(1687802781755.2136, 103344749371.00488),
 ('AMZN.O',
  'Price Close'): ('AMZN.O', 'Price Close') ~ N(162.4811904761905, 9.887612896554062),
 ('AMZN.O',
  '

In [133]:
data = (data - data.mean()) / data.std()
data.head()

AAPL.O                                          \
           1 Week Total Return    Volume Company Market Capitalization   
Date                                                                     
2024-01-02           -1.074963  1.372198                     -0.076901   
2024-01-03           -1.333217  0.040361                      -0.41574   
2024-01-04           -1.810063  0.791033                     -0.986161   
2024-01-05           -1.765312  0.259725                     -1.164112   
2024-01-08           -1.001736  0.080747                     -0.096403   

                                               AMZN.O            \
           Price Close Price Open 1 Week Total Return    Volume   
Date                                                              
2024-01-02   -0.118179   0.259153           -1.159567 -0.074113   
2024-01-03   -0.454729  -0.407055           -1.427879  0.077255   
2024-01-04   -1.021295   -0.87772           -2.187108  0.557197   
2024-01-05   -1.198044    -0.9141           -1.792625 -0.232752   
2024-01-08   -0.137549  -0.892499            -1.04152  -0.11637   

                                                                 ...  \
           Company Market Capitalization Price Close Price Open  ...   
Date                                                             ...   
2024-01-02                     -1.268372   -1.269385  -1.077521  ...   
2024-01-03                     -1.415058   -1.417045  -1.318565  ...   
2024-01-04                     -1.806889   -1.811478  -1.690431  ...   
2024-01-05                     -1.739574   -1.743716   -1.78314  ...   
2024-01-08                     -1.351762   -1.353329  -1.571969  ...   

                           WMT                                          \
           1 Week Total Return    Volume Company Market Capitalization   
Date                                                                     
2024-01-02            0.301966  0.113458                     -1.213416   
2024-01-03            -0.36477 -0.259841                     -1.211817   
2024-01-04            -0.91608 -0.218761                     -1.457978   
2024-01-05           -1.423505 -0.029363                     -1.625815   
2024-01-08            -0.73203 -0.109634                     -1.379654   

                                                  XOM            \
           Price Close Price Open 1 Week Total Return    Volume   
Date                                                              
2024-01-02   -1.200507  -1.448844           -0.132863  1.713984   
2024-01-03   -1.198944  -1.071694            0.351566  1.716857   
2024-01-04   -1.439715  -1.155505            0.568759  0.194899   
2024-01-05   -1.603877   -1.34857            0.761398 -1.130902   
2024-01-08   -1.363106  -1.531158            0.133306  1.672022   

                                                                 
           Company Market Capitalization Price Close Price Open  
Date                                                             
2024-01-02                      0.252895    0.226819  -0.344366  
2024-01-03                      0.600981    0.569179   0.193899  
2024-01-04                      0.236705    0.210895   0.915572  
2024-01-05                      0.362178    0.334304   0.552742  
2024-01-08                     -0.329946   -0.346436  -0.420122  

[5 rows x 125 columns]

In [134]:
# Now we process the lagged fields
ndata = data.copy()

In [135]:
# Construct the lagged fields
for ric in universe:
    for field_name in field_names:
        lag = fields[field_name].lag
        string_rep = fields[field_name].string_rep
        for cur_lag in range(1, lag+1):
            ndata[(ric, f'{string_rep}_lag_{cur_lag}')] = data[(ric, f'{string_rep}')].shift(cur_lag)
       
# Sort the lagged fields back into the correct order
ndata = ndata.sort_index(axis=1)
ndata.dropna(inplace=True)
ndata.head()

AAPL.O                                \
           1 Week Total Return Company Market Capitalization   
Date                                                           
2024-01-04           -1.810063                     -0.986161   
2024-01-05           -1.765312                     -1.164112   
2024-01-08           -1.001736                     -0.096403   
2024-01-09            0.122959                     -0.198786   
2024-01-10            0.566764                      0.057172   

                                                                              \
           Company Market Capitalization_lag_1 Price Close Price Close_lag_1   
Date                                                                           
2024-01-04                            -0.41574   -1.021295         -0.454729   
2024-01-05                           -0.986161   -1.198044         -1.021295   
2024-01-08                           -1.164112   -0.137549         -1.198044   
2024-01-09                           -0.096403    -0.23924         -0.137549   
2024-01-10                           -0.198786    0.014989          -0.23924   

                                                                           \
           Price Close_lag_2 Price Open Price Open_lag_1 Price Open_lag_2   
Date                                                                        
2024-01-04         -0.118179   -0.87772        -0.407055         0.259153   
2024-01-05         -0.454729    -0.9141         -0.87772        -0.407055   
2024-01-08         -1.021295  -0.892499          -0.9141         -0.87772   
2024-01-09         -1.198044  -0.475267        -0.892499          -0.9141   
2024-01-10         -0.137549  -0.377496        -0.475267        -0.892499   

                      ...                           XOM  \
              Volume  ... Company Market Capitalization   
Date                  ...                                 
2024-01-04  0.791033  ...                      0.236705   
2024-01-05  0.259725  ...                      0.362178   
2024-01-08  0.080747  ...                     -0.329946   
2024-01-09 -0.821151  ...                     -0.835884   
2024-01-10 -0.602568  ...                      -1.23254   

                                                                              \
           Company Market Capitalization_lag_1 Price Close Price Close_lag_1   
Date                                                                           
2024-01-04                            0.600981    0.210895          0.569179   
2024-01-05                            0.236705    0.334304          0.210895   
2024-01-08                            0.362178   -0.346436          0.334304   
2024-01-09                           -0.329946   -0.844053         -0.346436   
2024-01-10                           -0.835884   -1.234185         -0.844053   

                                                                           \
           Price Close_lag_2 Price Open Price Open_lag_1 Price Open_lag_2   
Date                                                                        
2024-01-04          0.226819   0.915572         0.193899        -0.344366   
2024-01-05          0.569179   0.552742         0.915572         0.193899   
2024-01-08          0.210895  -0.420122         0.552742         0.915572   
2024-01-09          0.334304  -0.196842        -0.420122         0.552742   
2024-01-10         -0.346436  -0.790926        -0.196842        -0.420122   

                                   
              Volume Volume_lag_1  
Date                               
2024-01-04  0.194899     1.716857  
2024-01-05 -1.130902     0.194899  
2024-01-08  1.672022    -1.130902  
2024-01-09  0.232614     1.672022  
2024-01-10 -0.246965     0.232614  

[5 rows x 275 columns]

In [136]:
ndata.to_csv('ndata.csv')

In [138]:
pred_horizon = 2 # we predict price close in 2 days

# Construct the target variable
for ric in universe:
    ndata[(ric, 'target')] = data[(ric, 'Price Close')].shift(-pred_horizon)
    
ndata = ndata.dropna()
ndata.head()

AAPL.O                                \
           1 Week Total Return Company Market Capitalization   
Date                                                           
2024-01-04           -1.810063                     -0.986161   
2024-01-05           -1.765312                     -1.164112   
2024-01-08           -1.001736                     -0.096403   
2024-01-09            0.122959                     -0.198786   
2024-01-10            0.566764                      0.057172   

                                                                              \
           Company Market Capitalization_lag_1 Price Close Price Close_lag_1   
Date                                                                           
2024-01-04                            -0.41574   -1.021295         -0.454729   
2024-01-05                           -0.986161   -1.198044         -1.021295   
2024-01-08                           -1.164112   -0.137549         -1.198044   
2024-01-09                           -0.096403    -0.23924         -0.137549   
2024-01-10                           -0.198786    0.014989          -0.23924   

                                                                           \
           Price Close_lag_2 Price Open Price Open_lag_1 Price Open_lag_2   
Date                                                                        
2024-01-04         -0.118179   -0.87772        -0.407055         0.259153   
2024-01-05         -0.454729    -0.9141         -0.87772        -0.407055   
2024-01-08         -1.021295  -0.892499          -0.9141         -0.87772   
2024-01-09         -1.198044  -0.475267        -0.892499          -0.9141   
2024-01-10         -0.137549  -0.377496        -0.475267        -0.892499   

                      ...    MSFT.O    NFLX.O    NVDA.O    ORCL.K        PG  \
              Volume  ...    target    target    target    target    target   
Date                  ...                                                     
2024-01-04  0.791033  ... -1.721373 -1.187649 -1.255019  -1.35925 -1.341894   
2024-01-05  0.259725  ... -1.644672  -1.24977 -1.165125 -1.587968 -1.209498   
2024-01-08  0.080747  ... -1.157969 -1.329216 -1.042496 -1.523572  -1.07059   
2024-01-09 -0.821151  ... -1.028275 -1.035518  -0.99466 -1.334824 -0.946876   
2024-01-10 -0.602568  ... -0.760518 -1.036997 -1.006011  -0.92846 -0.927342   

              TSLA.O       UNH         V       WMT       XOM  
              target    target    target    target    target  
Date                                                          
2024-01-04  1.886435  1.328667 -1.356328 -1.363106 -0.346436  
2024-01-05  1.596772  1.462006 -1.254489  -1.19738 -0.844053  
2024-01-08  1.542955  1.409391 -1.095928 -0.887818 -1.234185  
2024-01-09  1.188396  1.556424 -1.164251  -0.91596 -1.242147  
2024-01-10  0.748891  0.246822 -1.146203 -0.883128 -0.732587  

[5 rows x 300 columns]

In [ ]:
targets = ndata[[(ric, 'target') for ric in universe]]
targets.head()

,AAPL.O,AMZN.O,AVGO.O,BAC,BRKb,COST.O,CRM,GOOG.O,GOOGL.O,HD,...,MSFT.O,NFLX.O,NVDA.O,ORCL.K,PG,TSLA.O,UNH,V,WMT,XOM
,target,target,target,target,target,target,target,target,target,target,...,target,target,target,target,target,target,target,target,target,target
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-04,-0.137549,-1.353329,-1.528205,0.995797,-0.986378,-1.391117,-1.386701,-0.896159,-0.969197,-1.059366,...,-1.721373,-1.187649,-1.255019,-1.35925,-1.341894,1.886435,1.328667,-1.356328,-1.363106,-0.346436
2024-01-05,-0.23924,-1.123749,-1.439109,0.256423,-1.055872,-1.202558,-1.355075,-0.449651,-0.500428,-1.228049,...,-1.644672,-1.24977,-1.165125,-1.587968,-1.209498,1.596772,1.462006,-1.254489,-1.19738,-0.844053
2024-01-08,0.014989,-0.885066,-1.461499,0.214571,-1.000494,-1.010908,-1.167336,-0.176907,-0.204948,-0.199474,...,-1.157969,-1.329216,-1.042496,-1.523572,-1.07059,1.542955,1.409391,-1.095928,-0.887818,-1.234185
2024-01-09,-0.130285,-0.738418,-1.235144,-0.413199,-1.24915,-0.982744,-0.679484,-0.205501,-0.249381,-0.225649,...,-1.028275,-1.035518,-0.99466,-1.334824,-0.946876,1.188396,1.556424,-1.164251,-0.91596,-1.242147
2024-01-10,-0.050385,-0.795054,-1.145348,-0.901466,-1.233949,-0.650962,-0.642475,-0.080126,-0.122746,-0.305143,...,-0.760518,-1.036997,-1.006011,-0.92846,-0.927342,0.748891,0.246822,-1.146203,-0.883128,-0.732587


In [143]:
X = ndata.drop(columns=[(ric, 'target') for ric in universe])
y = targets
y.columns = y.columns.droplevel(1)

In [144]:
X.head()

AAPL.O                                \
           1 Week Total Return Company Market Capitalization   
Date                                                           
2024-01-04           -1.810063                     -0.986161   
2024-01-05           -1.765312                     -1.164112   
2024-01-08           -1.001736                     -0.096403   
2024-01-09            0.122959                     -0.198786   
2024-01-10            0.566764                      0.057172   

                                                                              \
           Company Market Capitalization_lag_1 Price Close Price Close_lag_1   
Date                                                                           
2024-01-04                            -0.41574   -1.021295         -0.454729   
2024-01-05                           -0.986161   -1.198044         -1.021295   
2024-01-08                           -1.164112   -0.137549         -1.198044   
2024-01-09                           -0.096403    -0.23924         -0.137549   
2024-01-10                           -0.198786    0.014989          -0.23924   

                                                                           \
           Price Close_lag_2 Price Open Price Open_lag_1 Price Open_lag_2   
Date                                                                        
2024-01-04         -0.118179   -0.87772        -0.407055         0.259153   
2024-01-05         -0.454729    -0.9141         -0.87772        -0.407055   
2024-01-08         -1.021295  -0.892499          -0.9141         -0.87772   
2024-01-09         -1.198044  -0.475267        -0.892499          -0.9141   
2024-01-10         -0.137549  -0.377496        -0.475267        -0.892499   

                      ...                           XOM  \
              Volume  ... Company Market Capitalization   
Date                  ...                                 
2024-01-04  0.791033  ...                      0.236705   
2024-01-05  0.259725  ...                      0.362178   
2024-01-08  0.080747  ...                     -0.329946   
2024-01-09 -0.821151  ...                     -0.835884   
2024-01-10 -0.602568  ...                      -1.23254   

                                                                              \
           Company Market Capitalization_lag_1 Price Close Price Close_lag_1   
Date                                                                           
2024-01-04                            0.600981    0.210895          0.569179   
2024-01-05                            0.236705    0.334304          0.210895   
2024-01-08                            0.362178   -0.346436          0.334304   
2024-01-09                           -0.329946   -0.844053         -0.346436   
2024-01-10                           -0.835884   -1.234185         -0.844053   

                                                                           \
           Price Close_lag_2 Price Open Price Open_lag_1 Price Open_lag_2   
Date                                                                        
2024-01-04          0.226819   0.915572         0.193899        -0.344366   
2024-01-05          0.569179   0.552742         0.915572         0.193899   
2024-01-08          0.210895  -0.420122         0.552742         0.915572   
2024-01-09          0.334304  -0.196842        -0.420122         0.552742   
2024-01-10         -0.346436  -0.790926        -0.196842        -0.420122   

                                   
              Volume Volume_lag_1  
Date                               
2024-01-04  0.194899     1.716857  
2024-01-05 -1.130902     0.194899  
2024-01-08  1.672022    -1.130902  
2024-01-09  0.232614     1.672022  
2024-01-10 -0.246965     0.232614  

[5 rows x 275 columns]

In [145]:
y.head()

,AAPL.O,AMZN.O,AVGO.O,BAC,BRKb,COST.O,CRM,GOOG.O,GOOGL.O,HD,...,MSFT.O,NFLX.O,NVDA.O,ORCL.K,PG,TSLA.O,UNH,V,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-04,-0.137549,-1.353329,-1.528205,0.995797,-0.986378,-1.391117,-1.386701,-0.896159,-0.969197,-1.059366,...,-1.721373,-1.187649,-1.255019,-1.35925,-1.341894,1.886435,1.328667,-1.356328,-1.363106,-0.346436
2024-01-05,-0.23924,-1.123749,-1.439109,0.256423,-1.055872,-1.202558,-1.355075,-0.449651,-0.500428,-1.228049,...,-1.644672,-1.24977,-1.165125,-1.587968,-1.209498,1.596772,1.462006,-1.254489,-1.19738,-0.844053
2024-01-08,0.014989,-0.885066,-1.461499,0.214571,-1.000494,-1.010908,-1.167336,-0.176907,-0.204948,-0.199474,...,-1.157969,-1.329216,-1.042496,-1.523572,-1.07059,1.542955,1.409391,-1.095928,-0.887818,-1.234185
2024-01-09,-0.130285,-0.738418,-1.235144,-0.413199,-1.24915,-0.982744,-0.679484,-0.205501,-0.249381,-0.225649,...,-1.028275,-1.035518,-0.99466,-1.334824,-0.946876,1.188396,1.556424,-1.164251,-0.91596,-1.242147
2024-01-10,-0.050385,-0.795054,-1.145348,-0.901466,-1.233949,-0.650962,-0.642475,-0.080126,-0.122746,-0.305143,...,-0.760518,-1.036997,-1.006011,-0.92846,-0.927342,0.748891,0.246822,-1.146203,-0.883128,-0.732587


## Train, Test, Validation Split

Now we have our `X` and we need to predict `y` values. We will split the data into training, testing and validation sets.